In [1]:
import numpy as np
from numpy import *
from random import shuffle
import math
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
def data_generate(M,P_PDG,W,N,K,Noise):
    #W是原数据集,代表哪些代理之间有连接，即网络
    #N是W的行数，即代理的个数
    #代理人的博弈策略
    player = zeros((M+1,N))
    #原始博弈策略放在第一行，随机设置
    for i in range(N):
        if random.random()<=0.5:
            player[0,i] = 1 #1代表合作
        else:
            player[0,i] = 0 #0代表不合作
    #计算每个节点的收益
    #M轮博弈
    F = zeros((1,N))
    G = zeros((M,N))
    A = zeros((N,M,N))
    for t in range(M):
        for i in range(N):
            if player[t,i] == 1:
                s1 = array([[1],[0]])
            else:
                s1 = array([[0],[1]])
            for j in range(N):
                if player[t,j] == 0:
                    s2 = array([[0],[1]])
                else:
                    s2 = array([[1],[0]])
                F[0,j] = ((s1.T).dot(P_PDG)).dot(s2) #如果代理i和j连接，则代理i的收益为F[0,j]
            A[i,t,:] = F  #A:N*M*N   
            # F是三维矩阵A的第i页，第t行
            G[t,i] = F.dot(W[:,i]) #第t轮代理i的收益
        # update strategies
        for k in range(N):
            s=[i for i,x in enumerate(list(W[:,k])) if x>=1]     # 找出与代理k合作的代理的索引
            if len(s)!=0: # 如果有代理与代理k合作
                shuffle(s)
                P = 1/(1+math.e**((G[t,k]-G[t,s[0]])/K)) # 费米规则
                if random.random()<= P:
                    player[t+1,k] = player[t,s[0]]
                else:
                    player[t+1,k] = player[t,k]
            else: # 如果没有代理与代理k合作
                player[t+1,k] = player[t,k]
    # add noise for G
    Aa = G + Noise*random.random((M,N)) # m轮收益总矩阵：包含每一轮的收益M*N
    return [A,Aa]

In [3]:
def TrainSTRidge(R, Ut, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [4]:
M = 10 #m轮更新
b = 1.2 #叛逃者的收益
K = 0.1
P_PDG = array([[1,0],[b,0]]) #2*2的收益矩阵
Noise = 5
W = np.loadtxt("dolphin.txt") 
T = W.shape[0] #全部社区内个体的总数量
SV = 6
# the part for generate EG data
y = zeros((SV*M,T))
AA = zeros((T,SV*M,T))
for i in range(SV): #产生6组A和Y的值
    [A, Aa] = data_generate(M,P_PDG,W,T,K,Noise)
    for j in range(M):
        y[i*M+j,:] =Aa[j,:]
        for k in range(T):
            AA[k,i*M+j,:] = A[k,j,:]

In [5]:
print(Noise*random.random((M,T)))

[[0.31973238 0.52472064 1.86106773 2.07928491 3.2040047  1.69889322
  4.99201074 0.87061827 0.23439416 1.92374683 2.28011678 2.65881852
  0.21263218 0.55987414 4.53596223 4.02509707 0.91219193 1.31270259
  1.86919857 4.47089115 3.10175461 3.35289012 3.5548755  0.14332207
  2.38588407 3.1122531  2.23726241 0.26295952 2.6061696  2.05958372
  0.54087098 0.77921968 3.59817394 3.99910005 3.04600515 2.73015595
  3.79145282 4.79656185 0.09264504 3.40377638 4.68505742 2.96094847
  1.65307252 1.98866877 4.28400648 3.58795702 0.84485239 3.81542351
  4.23468022 3.70083086 2.31197907 4.62061548 2.42934279 0.4185042
  4.47812766 3.12276248 3.54278346 0.62528617 3.25786322 0.5797584
  0.84708126 3.53925292]
 [1.91412929 4.01473863 0.7161044  0.45099011 3.34525978 2.65716681
  0.39252716 2.75093767 2.46460291 1.13370246 4.9964578  0.49776726
  3.73056816 2.54938299 2.03764584 0.24872115 0.49242682 4.01615032
  1.67879867 3.08958724 1.84913464 1.03770248 3.60877305 0.52825989
  3.80890356 1.88593239 1

In [11]:
X=zeros((T,T))
for j in range(T):
    w = TrainSTRidge(AA[j], y[:,j].reshape(-1,1),10**2,1)
    for i in range(w.shape[0]):
        X[i,j]=w[i,0]
np.savetxt("X.txt", X,fmt='%f',delimiter=' ')

<ipython-input-3-13ac3e2d7f51>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-3-13ac3e2d7f51>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
<ipython-input-11-0f2aa95661a7>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,j]=w[i,0]


In [12]:
def TrainSTRidge_SPL(R, Ut, v, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge_SPL.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge_SPL(R,Ut,v,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge_SPL(X0, y, v, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
    else: w = np.linalg.lstsq(X*v,y*v)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]*v) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y*v))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [72]:
X=zeros((T,T))
v = ones((SV*M,1))
v1 = zeros((SV*M,1))
Los = zeros((SV*M,T))
losss = zeros((SV*M,1))
lambda_0=2
t=0
while (t<30):  
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v,10**2,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        for i in range(SV*M):
            losss[i]=np.mean(Los[i,:])
        t = t+1
        for i in range(SV*M):
            if losss[i]<lambda_0:
                v[i]=1-losss[i]/lambda_0
            else:
                v[i]=0
        lambda_0 = lambda_0*1.2
        #lambda_0 = lambda_0 + np.mean(losss)      
np.savetxt("X1.txt", X,fmt='%f',delimiter=' ')

<ipython-input-12-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-12-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-72-d312225d135e>:18: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,j]=w[i,0]
<ipython-input-12-5e04d6e95ebc>:106: DeprecationWarning: elementwise comparison failed; this will raise an err

In [73]:
v

array([[0.9955816 ],
       [0.99618877],
       [0.99570593],
       [0.99559106],
       [0.99661468],
       [0.99606064],
       [0.996653  ],
       [0.99544602],
       [0.99535572],
       [0.99445454],
       [0.99654712],
       [0.99551444],
       [0.99660482],
       [0.9967245 ],
       [0.99655672],
       [0.99625789],
       [0.99557681],
       [0.99657966],
       [0.99604689],
       [0.99566116],
       [0.99627342],
       [0.99664808],
       [0.99669432],
       [0.99569287],
       [0.99637004],
       [0.99577093],
       [0.9962897 ],
       [0.9957412 ],
       [0.99544689],
       [0.99555285],
       [0.99638488],
       [0.99647795],
       [0.99521225],
       [0.99524348],
       [0.99498347],
       [0.99472013],
       [0.99505912],
       [0.99492707],
       [0.99516052],
       [0.99552895],
       [0.99587629],
       [0.99611192],
       [0.99567016],
       [0.99655976],
       [0.99594253],
       [0.99627597],
       [0.99604146],
       [0.996

In [74]:
X=zeros((T,T))
v = ones((SV*M,T))
v1 = zeros((SV*M,T))
Los = zeros((SV*M,T))
lambda_0=2
t=0
while (t<25):
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v[:,j].reshape(-1,1),10**2,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        t = t+1
        for j in range(T):
            for i in range(SV*M):
                if Los[i,j]<lambda_0:
                    v[i,j]=1-Los[i,j]/lambda_0
                else:
                    v[i,j]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X2.txt", X,fmt='%f',delimiter=' ')

<ipython-input-12-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-12-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-74-9b9da0bd393e>:17: ComplexWarning: Casting complex values to real discards the imaginary part
  X[i,j]=w[i,0]
<ipython-input-12-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine pr

In [75]:
v

array([[0.99360852, 0.99182599, 0.99306591, ..., 0.99611834, 0.9996592 ,
        0.99598325],
       [0.99269985, 0.98641488, 0.99110899, ..., 0.99333621, 0.99759457,
        0.99468098],
       [0.99941523, 0.98453523, 0.99335071, ..., 0.99822469, 0.99818597,
        0.99998158],
       ...,
       [0.99440547, 0.9996112 , 0.99340701, ..., 0.99369005, 0.99806001,
        0.99836005],
       [0.98805081, 0.99906964, 0.99990961, ..., 0.9889458 , 0.9988205 ,
        0.99472598],
       [0.99679979, 0.99669256, 0.99440044, ..., 0.9926274 , 0.99999685,
        0.99339232]])